In [2]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F


In [4]:
import torch
from torch_geometric.data import Data

num_nodes = 3  # Total number of nodes
num_train = 2  # Number of training nodes

# Creating masks:
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

# Manually specify which nodes are for training, validation, and testing
train_mask[:num_train] = True
val_mask[num_train:num_train + 1] = True  # Assuming one validation node
test_mask[num_train + 1:] = True  # Remaining nodes for testing

# Example node features and labels
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)  # Node features
y = torch.tensor([0, 1, 1], dtype=torch.long)  # Node labels
edge_index = torch.tensor([[0, 1], [1, 2]], dtype=torch.long)  # Edges

# Create Data object
data = Data(x=x, edge_index=edge_index.t().contiguous(), y=y,
            train_mask=train_mask, val_mask=val_mask, test_mask=test_mask)


In [6]:
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(data.num_node_features, 16)
        self.conv2 = GCNConv(16, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    # Apply the train mask to both output and labels
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

# Model instantiation, training loop, etc., remain similar to your existing code


In [8]:
for epoch in range(200):
    loss = train(data)
    print(f'Epoch {epoch}, Loss: {loss.item()}')

# Inference
model.eval()
with torch.no_grad():
    pred = model(data).argmax(dim=1)

print("Predictions:", pred)

Epoch 0, Loss: 0.7138559818267822
Epoch 1, Loss: 0.6754313707351685
Epoch 2, Loss: 0.6276906132698059
Epoch 3, Loss: 0.72585529088974
Epoch 4, Loss: 0.6592956781387329
Epoch 5, Loss: 0.7109458446502686
Epoch 6, Loss: 0.6202281713485718
Epoch 7, Loss: 0.6583640575408936
Epoch 8, Loss: 0.6474156975746155
Epoch 9, Loss: 0.736255407333374
Epoch 10, Loss: 0.6936805248260498
Epoch 11, Loss: 0.7524920105934143
Epoch 12, Loss: 0.7203025817871094
Epoch 13, Loss: 0.7215378284454346
Epoch 14, Loss: 0.6639000773429871
Epoch 15, Loss: 0.7205095887184143
Epoch 16, Loss: 0.6762562990188599
Epoch 17, Loss: 0.7077425718307495
Epoch 18, Loss: 0.7489963173866272
Epoch 19, Loss: 0.6936341524124146
Epoch 20, Loss: 0.7289021611213684
Epoch 21, Loss: 0.730404257774353
Epoch 22, Loss: 0.6792991757392883
Epoch 23, Loss: 0.7388244867324829
Epoch 24, Loss: 0.6787222623825073
Epoch 25, Loss: 0.7066164016723633
Epoch 26, Loss: 0.6661752462387085
Epoch 27, Loss: 0.7716382741928101
Epoch 28, Loss: 0.7701135277748108